In [1]:
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.film_id, i.inventory_id, r.rental_id, f.rental_rate, f.title, f.language_id, f.rental_duration, c.name AS 'category', r.rental_date AS rental_date,
f.length, f.replacement_cost, f.rating, f.special_features 
FROM sakila.rental r
JOIN sakila.inventory i
USING (inventory_id)
JOIN sakila.film f
USING (film_id)
JOIN sakila.film_category fc 
USING(film_id)
JOIN sakila.category c 
USING(category_id)
'''


sakila = pd.read_sql_query(query, engine)
sakila.head()

,film_id,inventory_id,rental_id,rental_rate,title,language_id,rental_duration,category,rental_date,length,replacement_cost,rating,special_features
0,19,93,10895,0.99,AMADEUS HOLY,1,6,Action,2005-08-02 01:16:59,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes"
1,19,93,12268,0.99,AMADEUS HOLY,1,6,Action,2005-08-18 04:26:54,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes"
2,19,94,3150,0.99,AMADEUS HOLY,1,6,Action,2005-06-20 20:35:28,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes"
3,19,94,5072,0.99,AMADEUS HOLY,1,6,Action,2005-07-09 05:01:58,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes"
4,19,94,9080,0.99,AMADEUS HOLY,1,6,Action,2005-07-30 08:02:39,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes"


In [3]:
sakila['rental_date_month'] = pd.DatetimeIndex(sakila['rental_date']).month

In [4]:
def get_rent_month(x):  
    
    if x == 5:
        return 'True'
    else:
        return 'False'
    
    
sakila['rented_last_month'] = sakila['rental_date_month'].apply(get_rent_month)
sakila.head()

,film_id,inventory_id,rental_id,rental_rate,title,language_id,rental_duration,category,rental_date,length,replacement_cost,rating,special_features,rental_date_month,rented_last_month
0,19,93,10895,0.99,AMADEUS HOLY,1,6,Action,2005-08-02 01:16:59,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",8,False
1,19,93,12268,0.99,AMADEUS HOLY,1,6,Action,2005-08-18 04:26:54,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",8,False
2,19,94,3150,0.99,AMADEUS HOLY,1,6,Action,2005-06-20 20:35:28,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",6,False
3,19,94,5072,0.99,AMADEUS HOLY,1,6,Action,2005-07-09 05:01:58,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",7,False
4,19,94,9080,0.99,AMADEUS HOLY,1,6,Action,2005-07-30 08:02:39,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",7,False


In [5]:
sakila['rented_last_month'].value_counts()

False    14888
True      1156
Name: rented_last_month, dtype: int64

In [6]:
sakila = sakila.drop('rental_date_month', axis=1)

In [7]:
y = sakila['rented_last_month'] #boolean/categorical
X = sakila.drop(['rented_last_month','title'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [8]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num = X_test.select_dtypes(include = np.number)

In [9]:
transformer = MinMaxScaler().fit(X_train_num)
X_train_norm = transformer.transform(X_train_num)
print(X_train_norm.shape)
X_train_num_scale = pd.DataFrame(X_train_norm, index = X_train_num.index, columns=X_train_num.columns)
X_train_num_scale.head()

(12835, 8)


,film_id,inventory_id,rental_id,rental_rate,language_id,rental_duration,length,replacement_cost
15993,0.980981,0.980786,0.238472,0.0,0.0,1.0,0.064748,0.20
14538,0.531532,0.532533,0.963796,1.0,0.0,0.5,0.244604,0.95
174,0.129129,0.129694,0.037014,0.0,0.0,1.0,0.460432,0.75
6083,0.965966,0.964192,0.471149,0.0,0.0,0.0,0.172662,0.25
9178,0.788789,0.788865,0.944479,0.5,0.0,1.0,0.237410,0.30


In [10]:
# use transformer do NOT fit again
X_test_norm = transformer.transform(X_test_num)
print(X_test_norm.shape)
X_test_num_scale = pd.DataFrame(X_test_norm, index = X_test_num.index, columns=X_test_num.columns)
X_test_num_scale.head()

(3209, 8)


,film_id,inventory_id,rental_id,rental_rate,language_id,rental_duration,length,replacement_cost
15533,0.338338,0.337336,0.694915,0.0,0.0,0.50,0.467626,0.40
13122,0.113113,0.112882,0.941862,0.0,0.0,0.00,0.107914,0.85
10796,0.657658,0.655240,0.898243,0.5,0.0,1.00,0.539568,0.50
3197,0.992993,0.993231,0.593843,0.5,0.0,0.75,0.949640,0.05
3084,0.863864,0.865284,0.753427,0.0,0.0,0.50,0.330935,0.60


In [30]:
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat = X_test.select_dtypes(include = object)
X_train_cat

,category,rating,special_features
15993,Travel,NC-17,Behind the Scenes
14538,Sports,PG,"Deleted Scenes,Behind the Scenes"
174,Action,PG-13,"Deleted Scenes,Behind the Scenes"
6083,Documentary,PG,Trailers
9178,Foreign,PG-13,"Trailers,Commentaries,Behind the Scenes"
...,...,...,...
3239,Classics,PG,"Trailers,Deleted Scenes"
9448,Games,PG-13,"Commentaries,Behind the Scenes"
860,Action,R,"Commentaries,Behind the Scenes"
8381,Foreign,NC-17,"Trailers,Commentaries"


In [31]:
r = pd.get_dummies(X_train_cat['rating'], prefix='rating')
r2= pd.get_dummies(X_test_cat['rating'], prefix='rating')

In [32]:
c = pd.get_dummies(X_train_cat['category'], prefix='genre')
c2 = pd.get_dummies(X_test_cat['category'], prefix='genre')

In [33]:
train_encoded = pd.concat([c,r],axis=1)
test_encoded = pd.concat([c2,r2],axis=1)

In [34]:
X_train_cat['special_features'] = X_train_cat['special_features'].str.split(',')
X_train_cat1 = X_train_cat.drop('special_features',1).join(X_train_cat.special_features.str.join('|').str.get_dummies())
X_train_cat1


<ipython-input-34-d6d1e4897fb6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_cat['special_features'] = X_train_cat['special_features'].str.split(',')


,category,rating,Behind the Scenes,Commentaries,Deleted Scenes,Trailers
15993,Travel,NC-17,1,0,0,0
14538,Sports,PG,1,0,1,0
174,Action,PG-13,1,0,1,0
6083,Documentary,PG,0,0,0,1
9178,Foreign,PG-13,1,1,0,1
...,...,...,...,...,...,...
3239,Classics,PG,0,0,1,1
9448,Games,PG-13,1,1,0,0
860,Action,R,1,1,0,0
8381,Foreign,NC-17,0,1,0,1


In [36]:
X_train_cat = pd.concat([train_encoded,X_train_cat1], axis=1)
X_train_cat = X_train_cat.drop(['category','rating'],axis=1)
X_train_cat

,genre_Action,genre_Animation,genre_Children,genre_Classics,genre_Comedy,genre_Documentary,genre_Drama,genre_Family,genre_Foreign,genre_Games,...,genre_Travel,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,Behind the Scenes,Commentaries,Deleted Scenes,Trailers
15993,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
14538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
174,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
6083,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
9178,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
9448,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,1,0,0
860,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
8381,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,1


In [ ]:
X_train_transformed = np.concatenate([X_train_num_scale, X_train_cat], axis=1)

In [ ]:
X_test_cat = X_test_cat.drop('special_features',1).join(X_test_cat.special_features.str.join('|').str.get_dummies())
X_test_cat.isna().sum()

In [ ]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [ ]:
X_train_cat = pd.concat([X_train_cat,train_encoded],axis=1)
X_test_cat = pd.concat([X_test_cat,test_encoded],axis=1)

In [ ]:
X_train_cat = X_train_cat.drop(['rating', 'category', 'film_title'], axis = 1)
X_train_cat

In [ ]:
X_test_cat = X_test_cat.drop(['rating', 'category', 'film_title'], axis = 1)
X_test_cat

In [ ]:
X_train = pd.concat([X_train_num_scale,X_train_cat], axis = 1)
X_test = pd.concat([X_test_num_scale,X_test_cat], axis = 1)

In [ ]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)